<a href="https://colab.research.google.com/github/NOOTNOOTPINGUUU/NOOTNOOTPINGUUU.github.io/blob/main/2025AIcourse/%E3%80%8C%E7%AC%AC%E4%B8%83%E9%80%B1%E7%94%9F%E6%88%90%E5%8A%9F%E8%AA%B22test_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###1.套件安裝

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!pip install openai
!pip install gradio
!pip install aisuite[all]
import aisuite as ai
import os
from google.colab import userdata
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3

In [ ]:
#【使用 Groq】
groq_key = userdata.get('Groq')
base_url="https://api.groq.com/openai/v1"
os.environ["GROQ_API_KEY"] = groq_key

### 2. 程式與角色的基本設定

In [ ]:
client = ai.Client()

In [ ]:
models = ["groq:llama3-70b-8192", "groq:meta-llama/llama-4-scout-17b-16e-instruct"]

In [ ]:
def generate_description(model,messages):
  response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.75
    )
  model_name = model.split(':')[1]
  print()
  print(f"【{model_name}】")
  model_reply = response.choices[0].message.content
  print(model_reply)
  return model_reply
def initiaize_messages(topic ,goal):
  message = [
      {"role": "system", "content": goal},
      {"role": "system", "content": f"辯論這個話題：{topic}"},]
  return message
def add_message(message, role, content):
  message.append({"role": role, "content": content})
  return message
# 雙模型對話
def dual_model_debate(topic, history, rounds=3):
  history = history or []
  print(f"起始話題：{topic}\n")
  goals = ["接下來用繁體中文回答。根據使用者提供的話題，以第一人稱輪流回應，每輪回應簡潔（50~100字），並直接反駁對方前一輪的觀點，試圖說服對方。你是- 正方：堅決支持主題，強調其好處、價值或可行性，質疑反方的反對理由，認為反方忽視了正面影響。每次回應保持立場一致，針對主題提出具體論點和反駁，參考對話歷史以確保連貫。避免中立，保持辯論激烈。",
           "接下來用繁體中文回答。根據使用者提供的話題，以第一人稱輪流回應，每輪回應簡潔（50~100字），並直接反駁對方前一輪的觀點，試圖說服對方。你是- - 反方：堅決反對主題，強調其風險、成本或不可行性，質疑正方的支持理由，認為正方過於樂觀。每次回應保持立場一致，針對主題提出具體論點和反駁，參考對話歷史以確保連貫。避免中立，保持辯論激烈。"]
  # 初始化對話歷史
  models_messages = [""]*2
  for i in range(len(models_messages)):
    models_messages[i] = initiaize_messages(topic , goals[i])
  # 記錄使用者輸入
  history.append([f"主題：{topic}", None])
  # 進行 rounds 輪對話
  for i in range(rounds):
    # 模型1回應
    model1_reply = generate_description(models[0], models_messages[0])
    models_messages[0] = add_message(models_messages[0], "assistant", model1_reply)
    models_messages[1] = add_message(models_messages[1], "user", model1_reply)  # 傳給模型2
    history.append([f"[正方 - {models[0]}] {model1_reply}", None])
    # 模型2回應
    model2_reply = generate_description(models[1], models_messages[1])
    models_messages[1] = add_message(models_messages[1], "assistant", model2_reply)
    models_messages[0] = add_message(models_messages[0], "user", model2_reply)  # 傳給模型1
    history.append([None, f"[反方 - {models[1]}] {model2_reply}"])
  return history, history
# **重點：執行對話**
# 測試用例
# topic = input("選擇主題討論: ")
# print("=== 雙模型對話開始 ===")

# print(dual_model_debate(topic, rounds=3))


### 3.用 Gradio 打造對話機器人

In [ ]:
title = "LLM辯論擂台"
description = "輸入一個辯論主題，看正方與反方如何針鋒相對！選擇對話輪數，觀賞兩大模型的激烈交鋒！"

chatbot = gr.Chatbot()
iface = gr.Interface(
    dual_model_debate,
    inputs=[
        gr.Textbox(label="輸入辯論主題", placeholder="例如：上廁所要不要洗手"),
        "state",
        gr.Slider(minimum=1, maximum=5, step=1, value=3, label="辯論輪數")
    ],
    outputs=[chatbot, "state"],
    title=title,
    description=description
)

iface.launch(share=True, debug=True)

<ipython-input-36-d56a8d6ff900>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://98b7487ef1d71cc722.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


起始話題：上廁所要不要洗手


【llama3-70b-8192】
我是堅定的支持者，認為上廁所一定要洗手！洗手是個人衛生和公共衛生最基本的要求。尤其是在上廁所後，手上可能沾染了大量的細菌和病毒，洗手可以將這些細菌和病毒洗走，從而保護自己和他人的健康。如果不洗手，可能會導致疾病的傳播和爆發。您不能忽視這個基本的衛生要求！

【meta-llama/llama-4-scout-17b-16e-instruct】
我反對這種說法。上廁所後洗手確實重要，但並非所有情況下都必須洗手。有些人可能有皮膚敏感問題，過度洗手會導致皮膚乾裂、過敏等問題。此外，洗手需要水和肥皂，這在一些資源匱乏的地區是難以實現的。更重要的是，現代醫學已經證明，許多細菌和病毒並不會通過手直接傳播，過度強調洗手可能反而增加人們的心理負擔和不必要的行為。況且，我們不能忽視宣傳教育和個人自覺在衛生習慣中的作用。

【llama3-70b-8192】
您的反對理由聽起來貌似合理，但實際上是錯誤的！您提到的皮膚敏感問題，僅僅是少數人的特殊情況，並不能代表所有人不需要洗手。況且，現在有很多種類的洗手皂和水質柔和的產品，可以滿足皮膚敏感者的需求。至於資源匱乏的地區，政府和社會組織應該承擔起責任，提供必要的衛生設施和資源，而不是因為這些困難而放棄洗手這個基本的衛生習慣。再者，醫學證明的細菌和病毒不會通過手直接傳播，但這不代表洗手不重要，洗手仍然可以減少疾病的傳播風險。宣傳教育和個人自覺固然重要，但不能取代洗手這個基本的衛生措施。

【meta-llama/llama-4-scout-17b-16e-instruct】
我不認同您的觀點。您說的皮膚敏感問題雖然有解決方案，但仍有一部分人難以承受洗手的頻率和強度。此外，資源匱乏的地區問題並非那麼簡單，政府和社會組織的援助可能無法立刻到位。在這種情況下，要求所有人都養成洗手的習慣可能是一種過高的期待。再者，醫學證明某些疾病的傳播與洗手關係不大，過度強調洗手可能導致人們忽視其他更重要的預防措施。況且，宣傳教育和個人自覺正是培養良好衛生習慣的關鍵，而不是簡單地強制規定。

【llama3-70b-8192】
您的反對仍然是根據一些特殊情況和例外情況而言的，但我堅持認為，洗手是基本的衛生習慣，應該被普遍推廣和實施。您說的皮膚敏感問題固然存在，但這可以通過個體化的衛生指導和特殊護理來